# CNN - Training - I

In this notebook we present the training process in convnets. In general, training a neural network means updating the network weights for minimizing the network loss. It is done via the gradient descent method, which is implemented by the **Backpropagation** algorithm. 

Convnet training, like training fully-connected multi-layer perceptrons (MLPs), consists of two phases:

- Forward propagation: The input signal is passed through the network. Output probabilities are produced, based on which the loss function is computed.
- Backward propagation: The gradient of the loss is calculated with respect to (w.r.t.) the network weights, the loss gradients are backpropagated and weights are updated.



We use the following simple convnet architecture to describe the training process. It consists of a multi-channel input (or input map), a convolution or conv layer (including activation layer), a pooling layer, a softmax layer (which includes a flattening layer at its beginning).


<img src="https://cse.unl.edu/~hasan/Pics/CNN_Simple_Architecture_1.png" width=600, height=300>

## Forward Propagation

The calculation of the forward signal is done for three layers, as shown in the figure below.
- Conv
- Pooling
- Softmax


<img src="https://cse.unl.edu/~hasan/Pics/CNN_Forward_Propagation.png" width=600, height=300>

### Forward Calculation: Conv and Pooling Layers

During the forward propagation, the multi-channel input (or input map) signal passes through the conv layer, followed by the max pooling layer. We use the previously defined **Conv_2D and Max_Pooling classes** for the forward calculation of the signal through these two layers. 



### Forward Calculation: Softmax Layer


Then, the output of the Max Pool layer is flattened and passed to the Softmax layer.

The Softmax layer computes $C$ probabilities for a given input for a $C$-class classification problem. We begin by calculating the output from the Softmax layer, i.e.,  $out$. Let's consider a simple 3-class classification scenario, as shown in the following figure (bottom). The $out$ signal is a vector of 3 probabilities (they must summed to 1). 

For a $d$-dimensional input signal instance $in$, the Softmax layer computes a score $s_c(in)$ for each $c$ class.

- $s_c(in) = w_c * in + b_c$ 

where $w_c$ and $b_c$ are the weights and bias associated with class $c$. For this 3-class classification problem, the weights of the Softmax layer is represented by a $d \times c$ matrix, and a single bias term for each class. 

There will be 3 scores $s_c(in)$ for 3 classes. Then, the Softmax layer estimates the probability of each class by applying the softmax function to the scores.

$p_c = \frac{exp(s_c)}{\sum_{c^{'}=1}^{C}exp(s_{c^{'}})}$

The softmax function computes the normalized exponential of the scores. This is also the output probability of class $c$, which we denote by:

$out(c) = \frac{exp(s_c)}{\sum_{c^{'}=1}^{C}exp(s_{c^{'}})}$


The output $out$ of the Softmax layer is a vector of predicted probabilities of length equivalent to the number of classes. 


<img src="https://cse.unl.edu/~hasan/Pics/CNN_Training_SoftmaxLayer.png" width=600, height=300>


The output $out$ will be used to compute the prediction loss. For that we define a differentiable loss function.


### Loss Function for Convnets

For C > 2 classes, each input signal will have one of the C labels. Thus, the input will have a multinomial distribution. The likelihood of a single input signal is defined as follows.

$Likelihood = \prod_{c=1}^{C}(p_c)^{y_c}$

Here $y_c$ is the true label of the signal. 


The goal of a convnet is to maximize the likelihood of the signals. However, instead of maximizing the likelihood, it is mathematically convenient to maximize its log-likelihood:

$Log\_Likelihood = \prod_{c=1}^{C}(p_c)^{y_c}$

=> $Log\_Likelihood = \sum_{c=1}^{C}y_clog(p_c)$

This is a candidate loss function. But its concave. We prefer to use a convex loss function, which is defined by using the negative of the log-likelihood (NLL) function:

$NLL= -\sum_{c=1}^{C}y_clog(p_c)$

The NLL is the **cross-entropy** function, which measures how well a set of estimated class probabilities match the target class labels. Thus, we have derived the network loss function $\mathcal{L}$ as the cross-entropy loss function.

$\mathcal{L}= -\sum_{c=1}^{C}y_clog(p_c)$


Note that an instance's class label is one-hot encoded. For a 3-class classification problem, each input will have 3 labels (0/1), one for each class. For each class label, a probability will be computed. The calculation of loss during forward propagation through the Softmax layer is illustrated below.

<img src="https://cse.unl.edu/~hasan/Pics/CNN_Training_Softmax_Forward_1.png" width=700, height=500>

For the 3-class classification example, the loss function can be written as:

$\mathcal{L}= - y_{c=1}log(p_{c=1}) - y_{c=2}log(p_{c=2}) - y_{c=3}log(p_{c=3})$



Note that for a given input signal only one of the true output label will be one, and rest will be zero. Thus, 
- the loss function includes the predicted probability associated with the true class label.

$\mathcal{L}= -log(p_c)$

This is not the function of the probabilities of all classes. It excludes the probabilities belonging to the incorrect class labels. 







## Backward Propagation

During backward propagation, the error signal propagates through the three layers of this network, in the reverse order, as shown in the figure below.
- Softmax
- Pooling
- Conv

<img src="https://cse.unl.edu/~hasan/Pics/CNN_Backward_Propagation.png" width=700, height=500>



For the backward calculation our goal is two-fold.

First, we need to compute the variation in loss due to the input signal of a layer, i.e., 
- $\frac{\partial \mathcal{L}}{\partial in}$

Each layer computes this based on the variation in loss due to the output signal that it receives from the next layer:
- $\frac{\partial \mathcal{L}}{\partial out}$

In other words:

$ \frac{\partial \mathcal{L}}{\partial in} \leftarrow \frac{\partial \mathcal{L}}{\partial out}$


<img src="https://cse.unl.edu/~hasan/Pics/CNN_Training_AnyLayer.png" width=500, height=400>

The second goal during backpropagation is to compute the variation in loss due to the layer weights and bias terms:
- $\frac{\partial \mathcal{L}}{\partial \vec{w}}$
- $\frac{\partial \mathcal{L}}{\partial b}$

Then, update the weights and the bias terms:

- $\vec{w} \leftarrow \vec{w} - \eta *  \frac{\partial \mathcal{L}}{\partial \vec{w}}$
- $b \leftarrow b - \eta *  \frac{\partial \mathcal{L}}{\partial b}$

Note that only the Softmax layer and the Conv layer has learnable weights. Thus the second goal of backpropagation applies to these two layers. The Max Pool layer only passes the $\frac{\partial \mathcal{L}}{\partial out}$ to the relevant neurons of the Conv layer.

We perform backward calculations for the following three layers.
- Backward Calculation: Softmax Layer
- Backward Calculation: Pooling Layer
- Backward Calculation: Conv Layer


## Backward Calculation: Softmax Layer

During the forward calculation, we derived the following loss function.

$\mathcal{L}= -log(p_c)$

The loss function includes the predicted probability associated with the true class label. It is not the function of the probabilities of all classes. It excludes the probabilities belonging to the incorrect class labels. Thus, its derivative with respect to the output vector of probabilities $out$ will have zeros for all classes except for the correct class.

$ \frac{\partial \mathcal{L}}{\partial out(c^{'})} =
  \begin{cases}
    0       & \quad \text{if } c^{'} \neq c\\
    -\frac{1}{p_c}  & \quad \text{if } c^{'} = c
  \end{cases}
$

We have computed the derivative that flows into the Softmax layer from the output. Using this, we will compute the derivative that leaves out of the Softmax layer, i.e., $\frac{\partial \mathcal{L}}{\partial in}$.


<img src="https://cse.unl.edu/~hasan/Pics/CNN_Training_SoftmaxLayer.png" width=600, height=300>

Using the chain rule of calculus, we write:

$\frac{\partial \mathcal{L}}{\partial in} = \frac{\partial \mathcal{L}}{\partial out(c)} \times \frac{\partial out(c)}{\partial s(c)} \times \frac{\partial s(c)}{\partial in}$


We have already computed the first term.

Let's derive the second term, i.e., the derivative of the class probabilities $out(c)$ w.r.t the class scores $s_c$. We will consider two cases.

- Case 1: $ \frac{\partial out(c)}{\partial s_c} $ if $c = c^{'}$

- Case 2: $ \frac{\partial out(c^{'})}{\partial s_c} $ if $c \neq c^{'}$

Let's calculate case 1 first.

$ \frac{\partial out(c)}{\partial s_c}  = \frac{exp(s_c)}{\sum_{c^{'}}exp(s_{c^{'}})} - (\frac{exp(s_c)}{\sum_{c^{'}}exp(s_{c^{'}})})^2$

We used the quotient rule of calculus: $(\frac{f}{g})^{'} = \frac{f^{'}g - fg^{'}}{g^2}$

$ \frac{\partial out(c)}{\partial s_c}  = \frac{exp(s_c)}{\sum_{c^{'}}exp(s_{c^{'}})}[1 - \frac{exp(s_c)}{\sum_{c^{'}}exp(s_{c^{'}})}]$

=> $ \frac{\partial out(c)}{\partial s_c}  = out(c) (1 - out(c))$

For case 2:

$ \frac{\partial out(c)}{\partial s_{c^{'}}} = -\frac{exp(s_c)*exp(s_{c^{'}})}{(\sum_{c^{'}}exp(s_{c^{'}}))^2}$ 

We used the product rule of calculus: $(fg)^{'} = f^{'}g + fg^{'}$

$ \frac{\partial out(c)}{\partial s_{c^{'}}} = -\frac{exp(s_c)}{\sum_{c^{'}}exp(s_{c^{'}})}*\frac{exp(s_{c^{'}})}{\sum_{c^{'}}exp(s_{c^{'}})}$

=> $ \frac{\partial out(c)}{\partial s_{c^{'}}} = -out(c) * out(c^{'})$

We combine both cases as follows.

$ \frac{\partial out(c)}{\partial s_{c^{'}}} =
  \begin{cases}
    out(c) (1 - out(c))   & \quad \text{if } c = c^{'} (case 1)\\
    -out(c) * out(c^{'})  & \quad \text{if } c \neq c^{'} (case 2)
  \end{cases}
$


Now compute the third term $\frac{\partial s(c)}{\partial in}$. We use the equation for computing the score: $s_c(in) = w_c * in + b_c$ 

By taking its derivative w.r.t. $in$, we obtain:

$\frac{\partial s(c)}{\partial in} = w_c$

The calculation of the derivative of loss w.r.t. the input of the Softmax layer, i.e., $\frac{\partial \mathcal{L}}{\partial in}$ is illustrated below. Note that the $\frac{\partial \mathcal{L}}{\partial out(c)}$ is non-zero only for the true class label for the given input signal. In the example below we assume that the true class label for the signal is $c = 2$. As a consequence, only  $\frac{\partial \mathcal{L}}{\partial out(c=2)}$ will contribute to the calculation of $\frac{\partial \mathcal{L}}{\partial in}$. 

- When we pass this loss derivative to the previous layer (Max Pool layer), we should reshape it using the shape of the input signal at the flatten layer. This is important because we used flattened input for the Softmax layer during the forward pass.

<img src="https://cse.unl.edu/~hasan/Pics/CNN_Training_Softmax_Backward.png" width=600, height=300>

The last part is to compute the variation in loss w.r.t. the weights and biases for updating the weights and the biases of the Softmax layer.

- $\vec{w} \leftarrow \vec{w} - \eta *  \frac{\partial \mathcal{L}}{\partial \vec{w}}$
- $b \leftarrow b - \eta *  \frac{\partial \mathcal{L}}{\partial b}$

Let's compute $\frac{\partial \mathcal{L}}{\partial \vec{w}}$:

$ \frac{\partial \mathcal{L}}{\partial \vec{w}} = \frac{\partial \mathcal{L}}{\partial out} \times \frac{\partial out}{\partial s(c)} \times \frac{\partial s(c)}{\partial \vec{w}} = \frac{\partial \mathcal{L}}{\partial out} \times \frac{\partial out}{\partial s(c)} \times in$ 

The last term is computed by taking the derivative of the score equation w.r.t. the weights $\vec{w}$. The first two terms are computed earlier.

Now compute $\frac{\partial \mathcal{L}}{\partial b}$:

$ \frac{\partial \mathcal{L}}{\partial b} = \frac{\partial \mathcal{L}}{\partial out} \times \frac{\partial out}{\partial s(c)} \times \frac{\partial s(c)}{\partial b} = \frac{\partial \mathcal{L}}{\partial out} \times \frac{\partial out}{\partial s(c)} \times 1$ 

Again, the last term is computed by taking the derivative of the score equation w.r.t. the bias term $b$.

That's it! We are done with the backward calculations for the Softmax layer.

## Backward Calculation: Max Pooling Layer

The Max Pooling layer doesn't have any trainable parameters (weights). That's why there is no training for this layer. However, we need to pass the $\frac{\partial \mathcal{L}}{\partial out}$ received from the Softmax layer to the Conv layer as $\frac{\partial \mathcal{L}}{\partial in}$. Note that this loss derivative doesn't change as it passes through the Max Pool layer, i.e.,

$\frac{\partial \mathcal{L}}{\partial in} = \frac{\partial \mathcal{L}}{\partial out}$ 

But how does this exactly work? The number of units in the Max Pool layer is smaller than the number of units in the Conv layer. Consider the following figure. The 4 x 4 Conv layer has 16 units, which are max pooled using a filter of pool size 2 x 2 with a stride of 2. As a result the size of the Max Pool layer is 2 x 2 with 4 units. 

Thus, although the Max Pool layer receives a 2 x 2 derivatives $\frac{\partial \mathcal{L}}{\partial out}$, it needs to pass a 4 x 4 derivatives $\frac{\partial \mathcal{L}}{\partial in}$. Also, it needs to find the exact location of the units in the Conv layer to pass the loss derivative where the original max value was in its corresponding Max Pool block. The other units in the Conv layer should receive zero loss derivative.


<img src="https://cse.unl.edu/~hasan/Pics/CNN_Training_MaxPooling_Forward.png" width=400, height=200>

Let's explain this. On the Conv layer only the max value pixels contribute to the Max Pool layer. The other pixel values have no effect on the Max Pool layer. Thus, these pixels don't influence the loss. Variation in these pixels have no impact on the loss. As a consequence, when we propagate the loss to the Conv layer we assign zero loss derivative to these Conv pixels $\frac{\partial \mathcal{L}}{\partial in} = 0$.


<img src="https://cse.unl.edu/~hasan/Pics/CNN_Training_MaxPooling_Backward.png" width=600, height=400>

Next we turn to the backward calculation in the Conv layer.